In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.0 MB/s eta 0:00:00


In [ ]:
!unzip -q "/content/Llava_Lama3.zip" -d "/content/"


/content/Llava_Lama3


In [ ]:
!pip install -r /content/Llava_Lama3/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [ ]:
import os
from pathlib import Path

HF_HOME = '/content/transformers_cache/huggingface'
HF_DATASETS_CACHE = '/content/transformers_cache/huggingface/datasets'
TRANSFORMERS_CACHE = '/content/transformers_cache/huggingface/models'

Path(HF_HOME).mkdir(parents=True, exist_ok=True)
Path(HF_DATASETS_CACHE).mkdir(parents=True, exist_ok=True)
Path(TRANSFORMERS_CACHE).mkdir(parents=True, exist_ok=True)

os.environ['HF_HOME'] = HF_HOME
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE
os.environ['TRANSFORMERS_CACHE'] = TRANSFORMERS_CACHE

In [ ]:
from datasets import load_dataset
from PIL import Image
from io import BytesIO
import requests
import os
import json
import uuid

def process_and_save(dataset, output_folder, subset_name):
    # Define image subfolder within output folder
    subset_folder = os.path.join(output_folder, subset_name)
    image_subfolder = os.path.join(output_folder, "images")

    if not os.path.exists(subset_folder):
        os.makedirs(subset_folder)

    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)

    # Initialize list to hold all JSON data
    json_data_list = []

    # Process and save images and labels
    for item in dataset:
        # Load image if it's a URL or a file path
        if isinstance(item["image"], str):
            response = requests.get(item["image"])
            image = Image.open(BytesIO(response.content))

        else:
            image = item["image"] # Assuming it's a PIL.Image object

        # Create a unique ID for each image
        unique_id = str(uuid.uuid4())

        # Define image path
        image_path = os.path.join(image_subfolder, f"{unique_id}.jpg")

        # Save image
        image.save(image_path)

        # Remove duplicates and format answers
        answers = item["answers"]
        unique_answers = list(set(answers))
        formatted_answers = ", ".join(unique_answers)

        # Structure for LLaVA JSON
        json_data = {
            "id": unique_id,
            "image": f"{unique_id}.jpg",
            "conversations": [
                {
                    "from": "human",
                    "value": item["question"]
                },
                {
                    "from": "gpt",
                    "value": formatted_answers
                }
            ]
        }

        # Append to list
        json_data_list.append(json_data)


    # Save the JSON data list to a file
    json_output_path = os.path.join(output_folder, subset_name, "dataset.json")
    with open(json_output_path, 'w') as json_file:
        json.dump(json_data_list, json_file, indent=4)

In [ ]:
def save_dataset(dataset_name, output_folder, class_name, subset_name, val_samples=None):
    # Load the dataset from Hugging Face
    dataset = load_dataset(dataset_name, split=subset_name)

    # Filter for images with the specified class in 'question_type'
    filtered_dataset = [item for item in dataset if item["question_type"] == class_name]

    # Determine the split for training and validation
    if val_samples is not None and subset_name == "train":
        train_dataset = filtered_dataset[val_samples:]
        val_dataset = filtered_dataset[:val_samples]

    else:
        train_dataset = filtered_dataset
        val_dataset = []

    # Process and save the datasets
    for subset, data in [("train", train_dataset), ("validation", val_dataset)]:
        if data:
            process_and_save(data, output_folder, subset)

In [ ]:
# Usage example
output_folder = 'dataset'
class_name = 'other'
val_samples = 300

save_dataset('Multimodal-Fatima/OK-VQA_train',
             output_folder,
             class_name,
             'train',
             val_samples)

save_dataset('Multimodal-Fatima/OK-VQA_test',
             output_folder,
             class_name,
             'test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5046 [00:00<?, ? examples/s]

In [ ]:
%cd /content/Llava_Lama3

/content/Llava_Lama3


In [ ]:
!pwd

/content/Llava_Lama3


In [ ]:
import os

output_dir = "/content/checkpoint/llava-llama-3-8b-qlora"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
from src.utils.hf_utils import re_direct_hf_cache
import wandb
from huggingface_hub.hf_api import HfFolder
from src.trainer_utils.train_arguments import *
from src.train import *

HF_HOME = '/content/transformers_cache/huggingface'
HF_DATASETS_CACHE = '/content/transformers_cache/huggingface/datasets'
TRANSFORMERS_CACHE = '/content/transformers_cache/huggingface/models'

re_direct_hf_cache(HF_HOME, HF_DATASETS_CACHE, TRANSFORMERS_CACHE)

wandb.login(key="YOUR WANDB API KEY")
HfFolder.save_token('YOUR HUGGING FAC API KEY')

output_dir = "../checkpoint/llava-llama-3-8b-qlora"

model_args = ModelArguments()
data_args = DataArguments()
training_args = TrainingArguments(output_dir)


model_args.model_name_or_path = "MohamedAhmedAE/Llama-3-8B-Instruct_pad_unk_tokens"
model_args.vision_tower = "openai/clip-vit-base-patch32"
data_args.data_path = "/content/dataset/train/dataset.json"
data_args.image_folder = "/content/dataset/images/"
data_args.validation_data_path = "/content/dataset/validation/dataset.json"
training_args.output_dir = output_dir
training_args.lora_enable = True
training_args.lora_r = 128
training_args.lora_alpha = 256
training_args.mm_projector_lr = 2e-5
training_args.bits = 4
model_args.version = "llama3"
model_args.mm_projector_type = "mlp2x_gelu"
model_args.mm_vision_select_layer = -2
model_args.mm_use_im_start_end = False
model_args.mm_use_im_patch_token = False
data_args.image_aspect_ratio = "pad"
training_args.group_by_modality_length = True
training_args.bf16 = False
training_args.fp16 = False
# training_args.num_train_epochs = 500
training_args.max_steps = 10
training_args.per_device_train_batch_size = 1
training_args.per_device_eval_batch_size = 1
training_args.gradient_accumulation_steps = 1
training_args.evaluation_strategy = "epoch"
training_args.save_strategy = "steps"
training_args.save_steps = 1
training_args.save_total_limit = 1
training_args.learning_rate = 2e-4
training_args.weight_decay = 0.
training_args.warmup_ratio = 0.03
training_args.lr_scheduler_type = "cosine"
training_args.logging_steps = 1
training_args.tf32 = False
training_args.model_max_length = 512
training_args.gradient_checkpointing = True
training_args.dataloader_num_workers = 2
data_args.lazy_preprocess = True
training_args.report_to = ["wandb"]
training_args.gradient_checkpointing_kwargs = {"use_reentrant": True}
training_args.push_to_hub = True
training_args.repo_id = "MohamedAhmedAE/llava-llama-3-8b-qlora"

train(model_args, data_args, training_args)

wandb: Currently logged in as: mohamed-ahmed. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_steps is given, it will override any value given in num_train_epochs


Total parameters: 4415736064
Trainable parameters: 355475456


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/p

Epoch,Training Loss,Validation Loss
0,0.852700,1.577204


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llama to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[2024-05-14 22:49:12,071] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llama to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
